In [1]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import glob
import openpyxl
import re
import csv
from google.cloud import bigquery

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Dados para subir\\\\Dados Brutos\\\\INEP\\\\Adequação Formação Docente')


In [3]:
df = pd.read_csv('INEP_adequacao_formacao_docente.csv', sep=';',encoding='latin-1')
df

,ano,nome_regiao,sigla_uf,id_municipio,nome_municipio,id_escola,nome_escola,localizacao,dependencia_adm,ei_grupo1,...,eja_ef_grupo1,eja_ef_grupo2,eja_ef_grupo3,eja_ef_grupo4,eja_ef_grupo5,eja_em_grupo1,eja_em_grupo2,eja_em_grupo3,eja_em_grupo4,eja_em_grupo5
0,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024275,CEEJA LUIZ VAZ DE CAMOES,Urbana,Estadual,NaN,...,"45,6","0,0","39,8","10,7","3,9","61,5","0,0","33,7","0,0","4,8"
2,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024291,EMMEF 07 DE SETEMBRO,Rural,Municipal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024313,EMMEF AFONSO AURELIO PORTO,Rural,Municipal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024372,EMEIEF ANA NERY,Rural,Municipal,"100,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1802862,2022,Centro-Oeste,DF,5300108,Brasília,53084020,ESC PEQUENO ENCANTO,Urbana,Privada,"100,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1802863,2022,Centro-Oeste,DF,5300108,Brasília,53084039,COL ANCHIETA,Urbana,Privada,"26,8",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1802864,2022,Centro-Oeste,DF,5300108,Brasília,53084055,COL MAPA,Urbana,Privada,"100,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1802865,2022,Centro-Oeste,DF,5300108,Brasília,53085000,COBIAN - COL BIANGULO,Urbana,Privada,"75,5",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = df.rename(columns= {'ef_total_grupo1': 'ef.total_grupo1', 
                        'ef_total_grupo2': 'ef.total_grupo2', 
                        'ef_total_grupo3': 'ef.total_grupo3', 
                        'ef_total_grupo4': 'ef.total_grupo4', 
                        'ef_total_grupo5': 'ef.total_grupo5',
                        'ef_finais_grupo1' : 'ef.finais_grupo1',
                        'ef_finais_grupo2' : 'ef.finais_grupo2', 
                        'ef_finais_grupo3' : 'ef.finais_grupo3', 
                        'ef_finais_grupo4' : 'ef.finais_grupo4',
                        'ef_finais_grupo5' : 'ef.finais_grupo5', 
                        'eja_ef_grupo1' : 'eja.ef_grupo1',
                        'eja_ef_grupo2' : 'eja.ef_grupo2', 
                        'eja_ef_grupo3' : 'eja.ef_grupo3',
                        'eja_ef_grupo4' : 'eja.ef_grupo4', 
                        'eja_ef_grupo5' : 'eja.ef_grupo5', 
                        'eja_em_grupo1' : 'eja.em_grupo1', 
                        'eja_em_grupo2' : 'eja.em_grupo2',
                        'eja_em_grupo3' : 'eja.em_grupo3', 
                        'eja_em_grupo4' : 'eja.em_grupo4', 
                        'eja_em_grupo5' : 'eja.em_grupo5', 
                        'ef_iniciais_grupo1' : 'ef.iniciais_grupo1',
                        'ef_iniciais_grupo2' : 'ef.iniciais_grupo2', 
                        'ef_iniciais_grupo3' : 'ef.iniciais_grupo3',
                        'ef_iniciais_grupo4' : 'ef.iniciais_grupo4',
                        'ef_iniciais_grupo5' : 'ef.iniciais_grupo5' })

In [6]:
df.columns

Index(['ano', 'nome_regiao', 'sigla_uf', 'id_municipio', 'nome_municipio',
       'id_escola', 'nome_escola', 'localizacao', 'dependencia_adm',
       'ei_grupo1', 'ei_grupo2', 'ei_grupo3', 'ei_grupo4', 'ei_grupo5',
       'ef.total_grupo1', 'ef.total_grupo2', 'ef.total_grupo3',
       'ef.total_grupo4', 'ef.total_grupo5', 'ef.iniciais_grupo1',
       'ef.iniciais_grupo2', 'ef.iniciais_grupo3', 'ef.iniciais_grupo4',
       'ef.iniciais_grupo5', 'ef.finais_grupo1', 'ef.finais_grupo2',
       'ef.finais_grupo3', 'ef.finais_grupo4', 'ef.finais_grupo5', 'em_grupo1',
       'em_grupo2', 'em_grupo3', 'em_grupo4', 'em_grupo5', 'eja.ef_grupo1',
       'eja.ef_grupo2', 'eja.ef_grupo3', 'eja.ef_grupo4', 'eja.ef_grupo5',
       'eja.em_grupo1', 'eja.em_grupo2', 'eja.em_grupo3', 'eja.em_grupo4',
       'eja.em_grupo5'],
      dtype='object')

In [7]:
df = df.melt(id_vars =['ano', 'nome_regiao', 'sigla_uf', 'id_municipio','nome_municipio','id_escola','nome_escola','localizacao','dependencia_adm'], 
        value_vars =['ei_grupo1', 'ei_grupo2', 'ei_grupo3', 'ei_grupo4', 'ei_grupo5',
       'ef.total_grupo1', 'ef.total_grupo2', 'ef.total_grupo3',
       'ef.total_grupo4', 'ef.total_grupo5', 'ef.iniciais_grupo1',
       'ef.iniciais_grupo2', 'ef.iniciais_grupo3', 'ef.iniciais_grupo4',
       'ef.iniciais_grupo5', 'ef.finais_grupo1', 'ef.finais_grupo2',
       'ef.finais_grupo3', 'ef.finais_grupo4', 'ef.finais_grupo5', 'em_grupo1',
       'em_grupo2', 'em_grupo3', 'em_grupo4', 'em_grupo5', 'eja.ef_grupo1',
       'eja.ef_grupo2', 'eja.ef_grupo3', 'eja.ef_grupo4', 'eja.ef_grupo5',
       'eja.em_grupo1', 'eja.em_grupo2', 'eja.em_grupo3', 'eja.em_grupo4',
       'eja.em_grupo5'], 
        var_name ='grupos_docentes_por_ano_escolar', 
        value_name ='per_valor')
df


,ano,nome_regiao,sigla_uf,id_municipio,nome_municipio,id_escola,nome_escola,localizacao,dependencia_adm,grupos_docentes_por_ano_escolar,per_valor
0,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,ei_grupo1,NaN
1,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024275,CEEJA LUIZ VAZ DE CAMOES,Urbana,Estadual,ei_grupo1,NaN
2,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024291,EMMEF 07 DE SETEMBRO,Rural,Municipal,ei_grupo1,NaN
3,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024313,EMMEF AFONSO AURELIO PORTO,Rural,Municipal,ei_grupo1,NaN
4,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024372,EMEIEF ANA NERY,Rural,Municipal,ei_grupo1,"100,0"
...,...,...,...,...,...,...,...,...,...,...,...
63100340,2022,Centro-Oeste,DF,5300108,Brasília,53084020,ESC PEQUENO ENCANTO,Urbana,Privada,eja.em_grupo5,NaN
63100341,2022,Centro-Oeste,DF,5300108,Brasília,53084039,COL ANCHIETA,Urbana,Privada,eja.em_grupo5,NaN
63100342,2022,Centro-Oeste,DF,5300108,Brasília,53084055,COL MAPA,Urbana,Privada,eja.em_grupo5,NaN
63100343,2022,Centro-Oeste,DF,5300108,Brasília,53085000,COBIAN - COL BIANGULO,Urbana,Privada,eja.em_grupo5,NaN


In [8]:
df['grupos_docentes_por_ano_escolar'].unique()

array(['ei_grupo1', 'ei_grupo2', 'ei_grupo3', 'ei_grupo4', 'ei_grupo5',
       'ef.total_grupo1', 'ef.total_grupo2', 'ef.total_grupo3',
       'ef.total_grupo4', 'ef.total_grupo5', 'ef.iniciais_grupo1',
       'ef.iniciais_grupo2', 'ef.iniciais_grupo3', 'ef.iniciais_grupo4',
       'ef.iniciais_grupo5', 'ef.finais_grupo1', 'ef.finais_grupo2',
       'ef.finais_grupo3', 'ef.finais_grupo4', 'ef.finais_grupo5',
       'em_grupo1', 'em_grupo2', 'em_grupo3', 'em_grupo4', 'em_grupo5',
       'eja.ef_grupo1', 'eja.ef_grupo2', 'eja.ef_grupo3', 'eja.ef_grupo4',
       'eja.ef_grupo5', 'eja.em_grupo1', 'eja.em_grupo2', 'eja.em_grupo3',
       'eja.em_grupo4', 'eja.em_grupo5'], dtype=object)

In [9]:
df1= df['grupos_docentes_por_ano_escolar'].str.split('_', n=1, expand=True)
df1

,0,1
0,ei,grupo1
1,ei,grupo1
2,ei,grupo1
3,ei,grupo1
4,ei,grupo1
...,...,...
63100340,eja.em,grupo5
63100341,eja.em,grupo5
63100342,eja.em,grupo5
63100343,eja.em,grupo5


In [11]:
df1.columns = ['ano_escolar', 'docente_grupos']
df1

,ano_escolar,docente_grupos
0,ei,grupo1
1,ei,grupo1
2,ei,grupo1
3,ei,grupo1
4,ei,grupo1
...,...,...
63100340,eja.em,grupo5
63100341,eja.em,grupo5
63100342,eja.em,grupo5
63100343,eja.em,grupo5


In [12]:
def transformar (nome):
    nome = re.sub(r"\bei\b","Ensino Infantil", nome)
    nome = re.sub(r"\bef.total\b","Ensino Fundamental - Todos os anos", nome)
    nome = re.sub(r"\bef.iniciais\b","Ensino Fundamental - Anos Iniciais", nome)
    nome = re.sub(r"\bef.finais\b","Ensino Fundamental - Anos Finais", nome)
    nome = re.sub(r"\beja.em\b","Ensino Médio do EJA", nome)
    nome = re.sub(r"\beja.ef\b","Ensino Fundamental do EJA", nome)
    nome = re.sub(r"\bgrupo1\b","Grupo 1", nome)
    nome = re.sub(r"\bgrupo2\b","Grupo 2", nome)
    nome = re.sub(r"\bgrupo3\b","Grupo 3", nome)
    nome = re.sub(r"\bgrupo4\b","Grupo 4", nome)
    nome = re.sub(r"\bgrupo5\b","Grupo 5", nome)
    return nome

In [14]:
df1['ano_escolar'] = df1['ano_escolar'].apply(transformar)

In [15]:
df1['docente_grupos'] = df1['docente_grupos'].apply(transformar)

In [16]:
df1

,ano_escolar,docente_grupos
0,Ensino Infantil,Grupo 1
1,Ensino Infantil,Grupo 1
2,Ensino Infantil,Grupo 1
3,Ensino Infantil,Grupo 1
4,Ensino Infantil,Grupo 1
...,...,...
63100340,Ensino Médio do EJA,Grupo 5
63100341,Ensino Médio do EJA,Grupo 5
63100342,Ensino Médio do EJA,Grupo 5
63100343,Ensino Médio do EJA,Grupo 5


In [17]:
df1['docente_grupos'].unique()

array(['Grupo 1', 'Grupo 2', 'Grupo 3', 'Grupo 4', 'Grupo 5'],
      dtype=object)

In [30]:
df1['ano_escolar'] = df1['ano_escolar'].replace('em', 'Ensino Médio')


In [32]:
df1['ano_escolar'].unique()

array(['Ensino Infantil', 'Ensino Fundamental - Todos os anos',
       'Ensino Fundamental - Anos Iniciais',
       'Ensino Fundamental - Anos Finais', 'Ensino Médio',
       'Ensino Fundamental do EJA', 'Ensino Médio do EJA'], dtype=object)

In [33]:
df = pd.concat([df, df1], axis=1)

In [34]:
df

,ano,nome_regiao,sigla_uf,id_municipio,nome_municipio,id_escola,nome_escola,localizacao,dependencia_adm,grupos_docentes_por_ano_escolar,per_valor,ano_escolar,docente_grupos
0,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,ei_grupo1,NaN,Ensino Infantil,Grupo 1
1,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024275,CEEJA LUIZ VAZ DE CAMOES,Urbana,Estadual,ei_grupo1,NaN,Ensino Infantil,Grupo 1
2,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024291,EMMEF 07 DE SETEMBRO,Rural,Municipal,ei_grupo1,NaN,Ensino Infantil,Grupo 1
3,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024313,EMMEF AFONSO AURELIO PORTO,Rural,Municipal,ei_grupo1,NaN,Ensino Infantil,Grupo 1
4,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024372,EMEIEF ANA NERY,Rural,Municipal,ei_grupo1,"100,0",Ensino Infantil,Grupo 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63100340,2022,Centro-Oeste,DF,5300108,Brasília,53084020,ESC PEQUENO ENCANTO,Urbana,Privada,eja.em_grupo5,NaN,Ensino Médio do EJA,Grupo 5
63100341,2022,Centro-Oeste,DF,5300108,Brasília,53084039,COL ANCHIETA,Urbana,Privada,eja.em_grupo5,NaN,Ensino Médio do EJA,Grupo 5
63100342,2022,Centro-Oeste,DF,5300108,Brasília,53084055,COL MAPA,Urbana,Privada,eja.em_grupo5,NaN,Ensino Médio do EJA,Grupo 5
63100343,2022,Centro-Oeste,DF,5300108,Brasília,53085000,COBIAN - COL BIANGULO,Urbana,Privada,eja.em_grupo5,NaN,Ensino Médio do EJA,Grupo 5


In [36]:
df = df.drop(columns='grupos_docentes_por_ano_escolar')

In [37]:
df.columns

Index(['ano', 'nome_regiao', 'sigla_uf', 'id_municipio', 'nome_municipio',
       'id_escola', 'nome_escola', 'localizacao', 'dependencia_adm',
       'per_valor', 'ano_escolar', 'docente_grupos'],
      dtype='object')

In [41]:
df = df[['ano', 'nome_regiao', 'sigla_uf', 'id_municipio', 'nome_municipio',
       'id_escola', 'nome_escola', 'localizacao', 'dependencia_adm','ano_escolar', 'docente_grupos', 'per_valor']]
df

,ano,nome_regiao,sigla_uf,id_municipio,nome_municipio,id_escola,nome_escola,localizacao,dependencia_adm,ano_escolar,docente_grupos,per_valor
0,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,Ensino Infantil,Grupo 1,NaN
1,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024275,CEEJA LUIZ VAZ DE CAMOES,Urbana,Estadual,Ensino Infantil,Grupo 1,NaN
2,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024291,EMMEF 07 DE SETEMBRO,Rural,Municipal,Ensino Infantil,Grupo 1,NaN
3,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024313,EMMEF AFONSO AURELIO PORTO,Rural,Municipal,Ensino Infantil,Grupo 1,NaN
4,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024372,EMEIEF ANA NERY,Rural,Municipal,Ensino Infantil,Grupo 1,"100,0"
...,...,...,...,...,...,...,...,...,...,...,...,...
63100340,2022,Centro-Oeste,DF,5300108,Brasília,53084020,ESC PEQUENO ENCANTO,Urbana,Privada,Ensino Médio do EJA,Grupo 5,NaN
63100341,2022,Centro-Oeste,DF,5300108,Brasília,53084039,COL ANCHIETA,Urbana,Privada,Ensino Médio do EJA,Grupo 5,NaN
63100342,2022,Centro-Oeste,DF,5300108,Brasília,53084055,COL MAPA,Urbana,Privada,Ensino Médio do EJA,Grupo 5,NaN
63100343,2022,Centro-Oeste,DF,5300108,Brasília,53085000,COBIAN - COL BIANGULO,Urbana,Privada,Ensino Médio do EJA,Grupo 5,NaN


In [46]:
df['per_valor'] = df['per_valor'].str.replace(',','.').astype(float)

Subindo para o GBQ

In [47]:
df.dtypes

ano                  int64
nome_regiao         object
sigla_uf            object
id_municipio         int64
nome_municipio      object
id_escola            int64
nome_escola         object
localizacao         object
dependencia_adm     object
ano_escolar         object
docente_grupos      object
per_valor          float64
dtype: object

In [42]:
client = bigquery.Client()

In [50]:
dataset_ref = client.dataset('percepcao_mensuracao_qualidade_estado')

In [51]:
schema = [bigquery.SchemaField('ano', 'INTEGER', description= 'Ano de referência da observação'),
          bigquery.SchemaField('nome_regiao', 'STRING', description= 'Região do Brasil abrangente a observação'),
          bigquery.SchemaField('sigla_uf', 'STRING', description= 'Sigla da Unidade da Federação referente a observação'),
          bigquery.SchemaField('id_municipio', 'INTEGER', description= 'Identificador do município pelo IBGE'),
          bigquery.SchemaField('nome_municipio', 'STRING', description= 'Nome do município da observação'),                    
          bigquery.SchemaField('id_escola', 'INTEGER', description= 'Identificação da escola'),                    
          bigquery.SchemaField('nome_escola', 'STRING', description= 'Nome da escola observada'),                    
          bigquery.SchemaField('localizacao', 'STRING', description= 'Se a área da observação é urbana ou rural'),
          bigquery.SchemaField('dependencia_adm', 'STRING', description= 'Dependência admnistrativa da observação (privada ou pública)'),
          bigquery.SchemaField('ano_escolar', 'STRING', description= 'Ano escolar referente ao qual docente leciona'),
          bigquery.SchemaField('docente_grupos', 'STRING', description= 'Cinco categorias de adequação da formação dos docentes em relação à disciplina que leciona'),
          bigquery.SchemaField('per_valor', 'FLOAT', description= 'Percentual de docências na respectiva unidade da agregação classificadas em cada uma das categorias'),
]

In [52]:
table_ref = dataset_ref.table('INEP_adequacao_formacao_docente')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=509cc866-b7f5-4733-b4dc-57e46b8794d5>